In [1]:
from load_data import *
from utils import *
from sklearn.decomposition import PCA
import numpy as np
import safeopt
import GPy

In [93]:
movie_features = load__movies_info()
user_features = load_users_info()
user_movie_ratings = load_user_movie_rating()

In [94]:
U_Mat = utility_matrix(user_movie_ratings)

In [154]:
movie_pca, movie_k = reduce_movie_features(movie_features, k=10)
user_pca, user_k = reduce_user_features(user_features, k=10)

In [155]:
movie_k
norm = np.linalg.norm(movie_k)
movie_k = movie_k/norm

In [156]:
norm = np.linalg.norm(user_k)
user_k = user_k/norm

In [163]:
std = 100

noise_var = 0.0001 ** 2

for user_idx, row in U_Mat.iterrows():
    ratings = np.nan_to_num(row.to_numpy())
    rated_idx = np.nonzero(ratings)
    
    parameters_set = []
    rating_look_up = {}
    for movie_idx in rated_idx[0]:
        v = movie_k[movie_idx].reshape(1, -1)
        u = user_k[user_idx - 1].reshape(1, -1)
        x = (v.T @ u).flatten()
        rating_look_up[x.sum()] = ratings[movie_idx]
        parameters_set.append(x)
    parameters_set = np.stack(parameters_set)

    init_safe_movie, init_reward = np.argmax(ratings), np.max(ratings)  
    v = movie_k[init_safe_movie].reshape(1, -1)
    u = user_k[user_idx - 1].reshape(1, -1)
    x0 = (v.T @ u).flatten()
    y0 = rating_look_up[x0.sum()]
    
    
#     gp = GPy.models.GPRegression(x0, y0, kernel, noise_var=noise_var)
#     kernel = GPy.kern.Linear(100, variances=std**2)
    kernel = GPy.kern.RBF(input_dim=100, variance=std**2, lengthscale=.4, ARD=True)
    likelihood = GPy.likelihoods.gaussian.Gaussian(variance=noise_var)
    gp = GPy.core.GP(np.array([x0]), np.array([[y0]]), kernel, likelihood)
    print(x0.sum())
    opt = safeopt.SafeOpt(gp, parameters_set, 2.5, lipschitz=None, threshold=2.5)
    for i in range(300):
        x_next = opt.optimize()
        reward = rating_look_up[x_next.sum()]
        print(reward, x_next.sum(), i)
        opt.add_new_data_point(x_next, reward)
    break
 

-0.00021592378935994218
5.0 -0.0012599692354270174 0
3.0 -0.00112833369374525 1
3.0 0.00027024076494717157 2
2.0 -0.001099773114702438 3
3.0 -0.00032338761149520767 4
1.0 -0.0001356599087559914 5
5.0 -0.0008727587261633484 6
4.0 -0.0002716033502031604 7
4.0 -0.0006917572381368458 8
4.0 -0.0012440027370155997 9
3.0 -0.0007027244606568723 10
1.0 -0.0009734205114290051 11
4.0 -0.0004543614311705575 12
5.0 -0.00038256663323858156 13
5.0 0.0005449497350594186 14
2.0 -0.0009781842704412959 15
5.0 2.2497934111278032e-05 16
5.0 -0.00038256663323858156 17
5.0 -0.00038256663323858156 18
5.0 2.2497934111278032e-05 19
5.0 -0.00038256663323858156 20
5.0 -0.00038256663323858156 21
5.0 -0.00038256663323858156 22
5.0 2.2497934111278032e-05 23
5.0 -0.00038256663323858156 24
5.0 -0.00038256663323858156 25
5.0 -0.00038256663323858156 26
5.0 2.2497934111278032e-05 27
5.0 -0.00038256663323858156 28
5.0 -0.00038256663323858156 29
5.0 2.2497934111278032e-05 30
5.0 -0.00038256663323858156 31
5.0 -0.0003825666

In [65]:
U_Mat

movie id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:

U_Mat.columns

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            1673, 1674, 1675, 1676, 1677, 1678, 1679, 1680, 1681, 1682],
           dtype='int64', name='movie id', length=1682)

In [13]:
U_Mat

movie id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
